# Methodology
> Explorations and explanations.

- toc: false 
- badges: true
- comments: false
- categories: [asot, bpm]
- image: images/chart-preview.png

In [17]:
#hide
import os
import yaml
import spotipy
import json 
from spotipy.oauth2 import SpotifyClientCredentials

with open('spotipy_credentials.yaml', 'r') as spotipy_credentials_file:
    credentials = yaml.safe_load(spotipy_credentials_file)
    os.environ["SPOTIPY_CLIENT_ID"] = credentials['spotipy_credentials']['spotipy_client_id']
    os.environ["SPOTIPY_CLIENT_SECRET"] = credentials['spotipy_credentials']['spotipi_client_seret']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

The Spotify Web API can return over a dozen [audio features for a track](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/), notably `tempo` - "The overall estimated tempo of a track in beats per minute (BPM)."

Given a Spotify ID, [Spotipy's `audio_features` method](https://spotipy.readthedocs.io/en/latest/?highlight=audio_features#spotipy.client.Spotify.audio_features) can be called as follows:

In [18]:
"""
Artist: Above & Beyond, Richard Bedford
Track: Sun & Moon
Track link: https://open.spotify.com/track/2CG1FmeprsyjgHIPNMYCf4
Track ID: 2CG1FmeprsyjgHIPNMYCf4
"""
sun_and_moon_id = '2CG1FmeprsyjgHIPNMYCf4'
audio_features = sp.audio_features(sun_and_moon_id)
print(json.dumps(audio_features, indent=2))

[
  {
    "danceability": 0.691,
    "energy": 0.522,
    "key": 6,
    "loudness": -8.024,
    "mode": 0,
    "speechiness": 0.0908,
    "acousticness": 0.0216,
    "instrumentalness": 0.0141,
    "liveness": 0.125,
    "valence": 0.187,
    "tempo": 133.995,
    "type": "audio_features",
    "id": "2CG1FmeprsyjgHIPNMYCf4",
    "uri": "spotify:track:2CG1FmeprsyjgHIPNMYCf4",
    "track_href": "https://api.spotify.com/v1/tracks/2CG1FmeprsyjgHIPNMYCf4",
    "analysis_url": "https://api.spotify.com/v1/audio-analysis/2CG1FmeprsyjgHIPNMYCf4",
    "duration_ms": 326267,
    "time_signature": 4
  }
]


Nice! Looks like the "`tempo`", or BPM, of this track is around 133. Let's continue.

Conveniently, the entire back catalogue of A State of Trance - 950+ episodes - has been uploaded to Spotify under the artist ["Armin van Buuren ASOT Radio"](https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce). [Spotipy's `artist_albums` method](https://spotipy.readthedocs.io/en/2.12.0/?highlight=audio_features#spotipy.client.Spotify.artist_albums) can list them for us, courtesy [spotipy/examples/artist_albums.py](https://github.com/plamere/spotipy/blob/2584d8cf5675ce877f773112a76d42fe36f8a1d1/examples/artist_albums.py#L29-L42):

In [19]:
"""
Artist: Armin van Buuren ASOT Radio
Artist link: https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce
Artist ID: 25mFVpuABa9GkGcj9eOPce
"""

asot_radio_id = '25mFVpuABa9GkGcj9eOPce'

albums = []
results = sp.artist_albums(asot_radio_id, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
seen = set()  # to avoid dups
for album in albums:
    name = album['name']
    if name not in seen:
        seen.add(name)

albums.sort(key=lambda x: x['release_date']) # Sort by release date

Cool, our list `albums` should now contain every episode of A State of Trance! Let's take a quick look..

In [20]:
# Print the names of the first 10 episodes
for album in albums[:10]:
    print(album['name'])

A State Of Trance Episode 001
A State Of Trance Episode 003
A State Of Trance Episode 004
A State Of Trance Episode 005
A State Of Trance Episode 007
A State Of Trance Episode 008
A State Of Trance Episode 009
A State Of Trance Episode 010
A State Of Trance Episode 012
A State Of Trance Episode 015


Hm, aren't we missing a few?

In [21]:
# How many episodes?
len(albums)

934

For some reason 25 early episodes are classified as "Singles and EPs". Let's grab those as well, and add them to the list.

In [22]:
"""
Artist: Armin van Buuren ASOT Radio
Artist link: https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce
Artist ID: 25mFVpuABa9GkGcj9eOPce
"""

asot_radio_id = '25mFVpuABa9GkGcj9eOPce'

singles = []
results = sp.artist_albums(asot_radio_id, album_type='single')
singles.extend(results['items'])
while results['next']:
    results = sp.next(results)
    singles.extend(results['items'])
seen = set()  # to avoid dups
for single in singles:
    name = single['name']
    if name not in seen:
        seen.add(name)

episodes = singles + albums

episodes.sort(key=lambda x: x['release_date']) # Sort by release date

for episode in episodes[:10]:
    print(episode['name'])

A State Of Trance Episode 000
A State Of Trance Episode 001
A State Of Trance Episode 002
A State Of Trance Episode 003
A State Of Trance Episode 004
A State Of Trance Episode 005
A State Of Trance Episode 006
A State Of Trance Episode 007
A State Of Trance Episode 008
A State Of Trance Episode 009


Nice!

In [23]:
# Now how many episodes?
len(episodes)

959

Great, that's every available episode as of writing. Let's see what we can do with all this, starting with a tracklist courtesy of [Spotipy's `album_tracks` method](https://spotipy.readthedocs.io/en/latest/?highlight=audio_features#spotipy.client.Spotify.album_tracks):

In [24]:
# Print every available Artist - Track from ASOT 001
for track in sp.album_tracks(episodes[1]['uri'])['items']:
    print(track['artists'][0]['name'], '-', track['name'])

Armin van Buuren - A State Of Trance [ASOT 001] - Intro
Liquid DJ Team - Liquidation [ASOT 001] - Marco V Mix
The Ultimate Seduction - The Ultimate Seduction [ASOT 001] **ASOT Radio Classic** - Original Mix
System F - Exhale [ASOT 001] - Ferry Corsten & Armin van Buuren New Mix
Rising Star - Clear Blue Moon [ASOT 001] - Original Mix
Ralphie B - Massive [ASOT 001] - Original Mix
Rank 1 - Such is Life [ASOT 001] - Original Mix
Armin van Buuren - Blue Fear [ASOT 001] - Original Mix
Armin van Buuren - A State Of Trance [ASOT 001] - Outro


Seems most of the early episodes are missing a bunch of tracks unfortunately, [A State of Trance's website reports twice as many tracks in this episode](http://www.astateoftrance.com/episodes/episode-001/) and we'll want to remove the intro and outro as well.

Looking at a more recent episode:

In [25]:
# Print every available Artist - Track from ASOT 950 - Part 2
for track in sp.album_tracks(episodes[945]['uri'])['items']:
    track_artist = track['artists'][0]['name']
    for artist in track['artists'][1:]:
        track_artist += " & " + artist['name']
    print(track_artist, '-', track['name'])

Armin van Buuren - A State Of Trance (ASOT 950 - Part 2) - Intro
Armin van Buuren - A State Of Trance (ASOT 950 - Part 2) - Coming Up, Pt. 1
Armin van Buuren & Avian Grays & Jordan Shaw & Cosmic Gate - Something Real (ASOT 950 - Part 2) [Tune Of The Week] - Cosmic Gate Remix
Armin van Buuren & BT & Nation Of One & Assaf - Always (ASOT 950 - Part 2) - Assaf Remix
Armin van Buuren & Inner City & Genix - It Could Be (ASOT 950 - Part 2) - Genix Remix
Spencer Brown & Qrion - Foggy August (ASOT 950 - Part 2) [Progressive Pick]
Spencer Brown & Liam Hathaway - LA ID (ASOT 950 - Part 2)
Armin van Buuren - A State Of Trance (ASOT 950 - Part 2) - Track Recap, Pt. 1
Armin van Buuren - A State Of Trance (ASOT 950 - Part 2) - Track Recap, Pt. 2
Armin van Buuren - A State Of Trance (ASOT 950 - Part 2) - ASOT 950 Moscow Line-up, Pt. 1
Armin van Buuren - A State Of Trance (ASOT 950 - Part 2) - ASOT 950 Moscow Line-up, Pt. 2
Kasablanca - Hold Me Close (ASOT 950 - Part 2)
Rolo Green - Kuat (ASOT 950 - Pa

The more recent episodes feature a Spotify exclusive - voiceover interludes! Seems they all contain "A State of Trance" though, same with the regular intros and outros.

Without them:

In [26]:
# Print every available Artist - Track from ASOT 950 - Part 2 (actual songs only)
episode_tracks = sp.album_tracks(episodes[945]['uri'])['items']
pruned_tracks = []

for track in episode_tracks:
    if "A State Of Trance".lower() in track['name'].lower() or "- Interview".lower() in track['name'].lower(): #ingore casing and interviews
        continue
    else:
        pruned_tracks.append(track)
        track_artist = track['artists'][0]['name']
        for artist in track['artists'][1:]:
            track_artist += " & " + artist['name']
        print(track_artist, '-', track['name'])

Armin van Buuren & Avian Grays & Jordan Shaw & Cosmic Gate - Something Real (ASOT 950 - Part 2) [Tune Of The Week] - Cosmic Gate Remix
Armin van Buuren & BT & Nation Of One & Assaf - Always (ASOT 950 - Part 2) - Assaf Remix
Armin van Buuren & Inner City & Genix - It Could Be (ASOT 950 - Part 2) - Genix Remix
Spencer Brown & Qrion - Foggy August (ASOT 950 - Part 2) [Progressive Pick]
Spencer Brown & Liam Hathaway - LA ID (ASOT 950 - Part 2)
Kasablanca - Hold Me Close (ASOT 950 - Part 2)
Rolo Green - Kuat (ASOT 950 - Part 2)
Farius & London Thor - Home Again (ASOT 950 - Part 2)
Above & Beyond - Blue Monday (ASOT 950 - Part 2)
Tom Fall - Guiding Light (ASOT 950 - Part 2)
MaRLo & Feenixpawl & SaberZ - Lighter Than Air (ASOT 950 - Part 2) [Trending Track] - SaberZ Remix
Andrew Rayel - Light Side Of The Harmony (FYH 200 Anthem) [ASOT 950 - Part 2]
Genix - Rave Daze (ASOT 950 - Part 2)
Protoculture - Titan (ASOT 950 - Part 2)
Giuseppe Ottaviani - Time To Play (ASOT 950 - Part 2)
BT - Atari's 

Much better! Finally, for fun, let's track this episode's BPM over time using some visualization libraries:

In [27]:
import altair as alt
import numpy as np
import pandas as pd

In [28]:
bpm = []
for track in pruned_tracks:
    bpm.append(sp.audio_features(track['uri'])[0]['tempo'])

x = np.arange(len(pruned_tracks))   

source = pd.DataFrame({
  'track': x,
  'bpm': np.array(bpm)
})

alt.Chart(source).mark_line().encode(
    alt.X('track'),
    alt.Y('bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="ASOT 950 Part 2 - BPM of track"
)

alt.Chart(...)

Not great, but it gets the point across.

Now, let's get exploring! ..